In [1]:
#getting all the packages
import os
import sys

print(f'os.environ={os.environ}')
print(f'sys.argv={sys.argv}')
print(f"MLFLOW_TRACKING_URI{os.environ.get('MLFLOW_TRACKING_URI')}")
print(f"sys.executable={sys.executable}")
print(f"sys.path={sys.path}")

os.environ=environ({'NV_LIBCUBLAS_DEV_VERSION': '11.4.1.1043-1', 'NV_CUDA_COMPAT_PACKAGE': 'cuda-compat-11-2', 'NV_CUDNN_PACKAGE_DEV': 'libcudnn8-dev=8.1.1.33-1+cuda11.2', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.8.4-1+cuda11.2', 'TCLLIBPATH': '/usr/share/tcltk/tcllib1.19', 'CLOUDSDK_PYTHON': 'python3', 'LANG': 'en_US.UTF-8', 'NV_LIBNPP_DEV_PACKAGE': 'libnpp-dev-11-2=11.3.2.152-1', 'HOSTNAME': '6520f85c6062', 'OLDPWD': '/', 'CLOUDSDK_CONFIG': '/content/.config', 'KMP_EXTRA_ARGS': '--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https://colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/m-s-1bxokgywbt81t --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true', 'NV_LIBNPP_VERSION': '11.3.2.152-1', 'NV_NVPROF_DEV_PACKAGE': 'cuda-nvprof-11-2=11.2.152-1', 'NVIDIA_VISIBLE_DEVI

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
import os
! rm -rf  logbert
! git clone https://github.com/HelenGuohx/logbert.git
! pwd

Cloning into 'logbert'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 130 (delta 9), reused 5 (delta 5), pack-reused 90
Receiving objects: 100% (130/130), 210.44 KiB | 7.01 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content


In [4]:
import sys
sys.path.append('/content/logbert/')
from loglizer.models import InvariantsMiner, PCA, IsolationForest, OneClassSVM, LogClustering, LR, SVM
from loglizer import dataloader, preprocessing
from loglizer.utils import metrics
from logdeep.dataset.session import sliding_window

In [5]:
#%cd logbert
import os
import gc
import pandas as pd
import numpy as np
import argparse
from tqdm import tqdm
import argparse
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

In [6]:
#train test split 
ouput_dir = "/content/gdrive/MyDrive/Windows/"
middle_dir = ""
log_file = "/content/gdrive/MyDrive/Windows/Windows_log_labeled.csv"

train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10
df=pd.read_csv(log_file)
whole_data_emb = df["Content"].values
whole_data_label=df["Label"].values

x_train, x_test, y_train, y_test = train_test_split(whole_data_emb, whole_data_label, test_size=test_ratio, random_state=1)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio))


#(x_train, y_train), (x_test, y_test) = dataloader.load_data(ouput_dir, middle_dir, log_file, is_mapping=True)
feature_extractor = preprocessing.FeatureExtractor()
x_train = feature_extractor.fit_transform(x_train)
x_test = feature_extractor.transform(x_test)

====== Transformed train data summary ======
Train data shape: 63972-by-86

====== Transformed test data summary ======
Test data shape: 7997-by-86



In [7]:
x_train[3]

array([1., 5., 3., 7., 2., 5., 1., 5., 2., 2., 3., 1., 0., 1., 3., 3., 1.,
       0., 4., 2., 2., 1., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [8]:
#PCA analysis
%%time
print("="*20 + " Model: PCA " + "="*20)
for th in np.arange(1):
    print("theshold", th)
    model = PCA(n_components=0.8, threshold=1, c_alpha = 1.9600)
    model.fit(x_train)
    print('Train validation:')
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Test validation:')
    precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: PCA ====================
theshold 0
====== Model summary ======
n_components: 1
Project matrix shape: 86-by-86
SPE threshold: 1

Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 4422, FP: 59550, TN: 0, FN: 0
Precision: 6.912%, recall: 100.000%, F1-measure: 12.931%

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 553, FP: 7444, TN: 0, FN: 0
Precision: 6.915%, recall: 100.000%, F1-measure: 12.936%

CPU times: user 548 ms, sys: 85 ms, total: 633 ms
Wall time: 557 ms


In [9]:
#isolation forest
%%time
print("="*20 + " Model: IsolationForest " + "="*20)
model = IsolationForest(n_estimators=100, max_samples='auto', contamination='auto', random_state=19)
model.fit(x_train)
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: IsolationForest ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======
Confusion Matrix: TP: 4422, FP: 4093, TN: 55457, FN: 0
Precision: 51.932, recall: 100.000, F1-measure: 68.362

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 553, FP: 487, TN: 6957, FN: 0
Precision: 53.173, recall: 100.000, F1-measure: 69.429

CPU times: user 8.11 s, sys: 184 ms, total: 8.3 s
Wall time: 8.3 s


In [10]:
#SVM
%%time
print("="*20 + " Model: one class SVM " + "="*20)
model = OneClassSVM(kernel='rbf')
model.fit(x_train, y_train)

print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: one class SVM ====================
====== Model summary ======
Train validation:
====== Evaluation summary ======


/content/logbert/loglizer/utils.py:37: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)


Confusion Matrix: TP: 0, FP: 59550, TN: 0, FN: 4422
Precision: 0.000, recall: 0.000, F1-measure: nan

Test validation:
====== Evaluation summary ======
Confusion Matrix: TP: 0, FP: 7444, TN: 0, FN: 553
Precision: 0.000, recall: 0.000, F1-measure: nan

CPU times: user 15min 52s, sys: 1.32 s, total: 15min 53s
Wall time: 15min 53s


/content/logbert/loglizer/utils.py:37: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * precision * recall / (precision + recall)


In [11]:
#Clustering
%%time
print("="*20 + " Model: LogClustering " + "="*20)
max_dist = 0.3  # the threshold to stop the clustering process
anomaly_threshold = 0.3  # the threshold for anomaly detection
model = LogClustering(max_dist=max_dist, anomaly_threshold=anomaly_threshold)
model.fit(x_train[y_train == 0, :])  # Use only normal samples for training
print('Train validation:')
precision, recall, f1 = model.evaluate(x_train, y_train)
print('Test validation:')
precision, recall, f1 = model.evaluate(x_test, y_test)

==================== Model: LogClustering ====================
====== Model summary ======
Starting offline clustering...
Processed 1000 instances.
Found 7 clusters offline.

Starting online clustering...
Processed 2000 instances.
Processed 4000 instances.
Processed 6000 instances.
Processed 8000 instances.
Processed 10000 instances.
Processed 12000 instances.
Processed 14000 instances.
Processed 16000 instances.
Processed 18000 instances.
Processed 20000 instances.
Processed 22000 instances.
Processed 24000 instances.
Processed 26000 instances.
Processed 28000 instances.
Processed 30000 instances.
Processed 32000 instances.
Processed 34000 instances.
Processed 36000 instances.
Processed 38000 instances.
Processed 40000 instances.
Processed 42000 instances.
Processed 44000 instances.
Processed 46000 instances.
Processed 48000 instances.
Processed 50000 instances.
Processed 52000 instances.
Processed 54000 instances.
Processed 56000 instances.
Processed 58000 instances.
Processed 59550 